In [50]:
parent_path='../Data/New_Data_15-06-2020/'
%load_ext autoreload
%autoreload 2
import parmap
import pandas as pd
from tqdm import tqdm,tqdm_notebook
from models.tokenization import *
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from models.model_architecture import *
from models.train_eval import *
from models.model_utils import *
from models.tokenization import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
whatsapp_data=pd.read_csv(parent_path+'Data_text_spam_removed_v02.csv')
temp=whatsapp_data[['group_id_anonymized','phone_num_anonymized','message_text','timestamp']]
duplicateDFRow = temp[temp.duplicated()]
whatsapp_data=whatsapp_data.drop(list(duplicateDFRow.index))

In [52]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')

all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

data_fear_speech=whatsapp_data[whatsapp_data['orig_index'].isin(all_fear_speech_index)]
data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]



temp_data=whatsapp_data[whatsapp_data['group_id_anonymized'].isin(data_fear_speech.group_id_anonymized.unique())]

100%|██████████| 4782/4782 [00:00<00:00, 12214.69it/s]


In [39]:
n_process=30
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}

def preprocess_split(df):
    preprocessed_sents=[preprocess_sent(ele,params=params_preprocess) for ele in tqdm(df['message_text'],total=len(df))]
    df['preprocessed']=preprocessed_sents
    return df


df_split = np.array_split(temp_data, n_process)
list_all = parmap.map(preprocess_split, df_split,pm_processes=n_process,pm_pbar=True)


100%|██████████| 30/30 [03:16<00:00,  6.54s/it]


In [40]:
import gc
gc.collect()

97

In [41]:
temp_data=pd.concat(list_all,axis=0)
temp_data

,group_id_anonymized,phone_num_anonymized,message_text,timestamp,media_type,media_url,forwarded,orig_index,language,only_urls,is_spam,preprocessed
2,9,9,Dilli me aaj ole,1549559506000,None,None,0,10,et,0,0,dilli me aaj ole
3,9,9,Me gajendra dilli,1549559512000,None,None,0,11,sq,0,0,me gajendra dilli
5,21,18,*📰FR62* *👉दिल्ली-एनसीआर में शिमला जैसी बर्फबार...,1549559738000,None,None,0,26,hi,0,0,fr दिल्लीएनसीआर में शिमला जैसी बर्फबारी देखें...
6,24,22,कल मैने एक आदमी से लिफ्ट माँगा उसके बाद धन्यवा...,1549559843000,None,None,1,31,hi,0,0,कल मैने एक आदमी से लिफ्ट माँगा उसके बाद धन्यवा...
9,28,25,*📰FR62* *👉दिल्ली-एनसीआर में शिमला जैसी बर्फबार...,1549559902000,None,None,0,36,hi,0,0,fr दिल्लीएनसीआर में शिमला जैसी बर्फबारी देखें...
...,...,...,...,...,...,...,...,...,...,...,...,...
2501723,9952,70753,Mallepally Mohd Yousuf Bhai and Team Met AIMIM...,1535372562000,None,None,65,6332493,en,0,0,mallepally mohd yousuf bhai and team met aimim...
2501724,9952,70753,Must Watch and Share Uno To Nahi Aanewale Mai...,1536263726000,None,https://www.facebook.com/story.php?story_fbid=...,64,6332494,en,0,0,must watch and share uno to nahi aanewale mait...
2501725,9952,70753,Must Watch and Share Uno To Nahi Aanewale Mai...,1536322463000,None,https://www.facebook.com/story.php?story_fbid=...,64,6332495,en,0,0,must watch and share uno to nahi aanewale mait...
2501726,9952,70753,Pic From GHMC Council Hall At Tank Bund On 25 ...,1535269807000,None,None,65,6332496,en,0,0,pic from ghmc council hall at tank bund on aug...


In [ ]:
params={'model_path':'xlm-roberta-base',
        'preprocess_doc':False,
        'max_length':256,
        'batch_size':32,
        'hidden_size':128,
        'weights':[1.0,3.0],
        'seq_model':'lstm',
        'data_path':parent_path+'Fearspeech_data_final.pkl',
        'max_sentences_per_doc':5,
        'transformer_type':'normal_transformer',
        'take_tokens_from':'both',
        'device':'cpu',
        'learning_rate':2e-5,
        'epsilon':1e-8,
        'random_seed':2020,
        'epochs':10,
        'max_memory':0.6,
        'freeze_bert':False
       }


In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


def inference_phase(params,temp,fold):
    params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
    if(params['preprocess_doc']==True):
        list_sents=[preprocess_doc(ele,params=params_preprocess) for ele in tqdm(temp['message_text'],total=len(temp))]
    else:
        list_sents=[preprocess_sent(ele,params=params_preprocess) for ele in tqdm(temp['message_text'],total=len(temp))]
    X_0 = np.array(list_sents,dtype='object')
    y_0 = np.array([0]*len(list_sents))
    tokenizer = AutoTokenizer.from_pretrained(params['model_path'])
    if torch.cuda.is_available() and params['device']=='cuda':    
        # Tell PyTorch to use the GPU.    
        device = torch.device("cuda")
        deviceID = get_gpu(params)
        torch.cuda.set_device(deviceID[0])
    else:
        print('Since you dont want to use GPU, using the CPU instead.')
        device = torch.device("cpu")
    X_test,X_test_mask= encode_sent(X_0,params,tokenizer)
    test_dataloader=return_dataloader_sent(X_test,X_test_mask,y_0,params,is_train=True)
    #params['model_path']='Saved/'+params['model_path']+'_'+str(fold)
    params['load_saved']=True
    model=select_transformer_model(params['transformer_type'],params['model_path'],params,fold)
    model.eval()    
    logits_all=[]
    # Evaluate data for one epoch
    for batch in tqdm(test_dataloader,total=len(test_dataloader)):
        # Add batch to GPU
        b_input_ids = batch[0].to(device)
        b_att_masks = batch[1].to(device)
        b_token_types = batch[2].to(device)
        b_labels = batch[3].to(device)
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            outputs = model(input_ids=b_input_ids, attention_mask=b_att_masks, token_type_ids=b_token_types)

        logits = outputs
        logits = logits.detach().cpu().numpy()
        
        
        logits_all+=list(logits)
        
    logits_all_final=[]
    for logits in logits_all:
        logits_all_final.append(softmax(logits)[1])
    
    return np.array(logits_all_final)

In [ ]:
for i in range(1,6):
    pred_labels=inference_phase(params,temp,1)
    temp['fold_'+str(i)]=list(pred_labels)

In [ ]:
temp

In [75]:
whatsapp_data_pred=pd.read_csv(parent_path+'Data_text_spam_removed_v02_preds.csv')


In [54]:
lexicon_path='../Data/Important_lexicon/'
########################################
def get_lexicon(type1,path):
    lexicon=[]
    with open(path+type1+'_keywords_complete.txt', 'r') as f:
        temp=f.read().splitlines()
    for word in temp:
        lexicon.append(word.split('_'))
    return lexicon

muslim_lexicon=get_lexicon('muslim',lexicon_path)
hindu_lexicon=get_lexicon('hindu',lexicon_path)


In [43]:
from cltk.tokenize.word import WordTokenizer


# Cell
tok = WordTokenizer(language='multilingual')

def add_keywords(df1,lexicon,type_):
    occ_list=[]
    phrase_present=[]
    count_empty=0
    for index,row in tqdm_notebook(df1.iterrows(),total=len(df1)):
        temp=[]
        try:
            x=tok.tokenize(row['preprocessed'])
        except:
            x= ["dummy"]
        for y in lexicon:
            occ = [i for i,a in enumerate(x) if a == y[0]]
            for b in occ:
                  if x[b:b+len(y)] == y:
                    temp.append([b,b+len(y)])
        if(len(temp)>0):
            temp.sort(key=lambda interval: interval[0])
            merged = [temp[0]]
            for current in temp:
                previous = merged[-1]
                if current[0] <= previous[1]:
                    previous[1] = max(previous[1], current[1])
                else:
                    merged.append(current)
            occ_list.append(merged)
            phrase_present.append(len(merged))
        else:
            occ_list.append([])
            phrase_present.append(0)
            count_empty+=1
    df1[type_+'_phrases_count']=phrase_present
    #df1[type_+'_phrases_to_check']=occ_list
    return df1    
        

W1009 08:37:32.026781 140342880130880 word.py:92] Falling back to default tokenizer, the NLTK's `TreebankWordTokenizer()`.


In [45]:
#filtered_whatsapp_data=add_keywords(whatsapp_data_pred,hindu_lexicon,'hindu')
temp_data=add_keywords(temp_data,muslim_lexicon,'muslim')

/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [49]:
temp_data[temp_data['muslim_phrases_count']>0][['message_text']].sample(1000).to_csv('../muslim.csv',index=False)

In [27]:
whatsapp_data_pred_2=filtered_whatsapp_data[(filtered_whatsapp_data['muslim_phrases_count']>0)]
whatsapp_data_pred_2.to_csv(parent_path+'Data_text_spam_removed_v02_preds.csv')

NameError: name 'filtered_whatsapp_data' is not defined

In [110]:
whatsapp_data_pred=pd.read_csv(parent_path+'Data_text_spam_removed_v03_preds.csv')

In [111]:
len(whatsapp_data_pred)

118627

In [112]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

whatsapp_data_pred=whatsapp_data_pred[~whatsapp_data_pred['orig_index'].isin(all_fear_speech_index)]
#data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]


100%|██████████| 4782/4782 [00:00<00:00, 15025.72it/s]


In [113]:
len(whatsapp_data_pred)

110923

In [79]:
whatsapp_data_pred = whatsapp_data_pred[(whatsapp_data_pred['muslim_phrases_count']>0)]

In [80]:
whatsapp_data_pred

,group_id_anonymized,phone_num_anonymized,message_text,timestamp,media_type,media_url,forwarded,orig_index,language,only_urls,is_spam,preprocessed,hindu_phrases_count,muslim_phrases_count,fold_1,fold_2,fold_3,fold_4,fold_5
7,105,119,राहुल गांधी मुसलमान हैं केजरीवाल बेईमान हैं मायावती जातिवादी हैं ओर चौकीदार ही चोर हैँ! ठीक,1549564154000,None,None,0,279,hi,0,0,राहुल गांधी मुसलमान हैं केजरीवाल बेईमान हैं मायावती जातिवादी हैं ओर चौकीदार ही चोर हैँ ठीक,0,1,0.015219,0.860916,0.005268,0.229787,0.021000
8,113,128,*इरशाद अली* *सदस्य समाजवादी पार्टी उo प्रo*,1549564724000,None,None,0,346,ne,0,0,इरशाद अली सदस्य समाजवादी पार्टी उo प्रo,0,1,0.249272,0.007510,0.003237,0.066088,0.042949
9,115,128,*इरशाद अली* *सदस्य समाजवादी पार्टी उo प्रo*,1549564733000,None,None,0,348,ne,0,0,इरशाद अली सदस्य समाजवादी पार्टी उo प्रo,0,1,0.033778,0.010212,0.025451,0.043653,0.026955
10,112,128,*इरशाद अली* *सदस्य समाजवादी पार्टी उo प्रo*,1549564745000,None,None,0,349,ne,0,0,इरशाद अली सदस्य समाजवादी पार्टी उo प्रo,0,1,0.027885,0.006348,0.864001,0.222577,0.038866
11,16,129,"🌴 पैगाम 🌲🌲 एक बच्चा जला देने वाली गर्मी में नंगे पैर गुलदस्ते बेच रहा था लोग उसमे भी मोलभाव कर रहे थे। ☘एक आदमी को उसके पैर देखकर बहुत दुःख हुआ, उस आदमी ने बाज़ार से नया जूता ख़रीदा और उसे देते हुए कहा ""बेटा लो, ये जूता पहन लो"" . 🌸लड़के ने फ़ौरन जूते निकाले और पहन लिए . 🌼उसका चेहरा ख़ुशी से दमक उठा था. 🍀वो उस आदमी की तरफ़ पल्टा और हाथ थाम कर पूछा, ""आप अल्लाह हैं? . 🌻""उसने घबरा कर हाथ छुड़ाया और कानों को हाथ लगा कर कहा, ""नहीं बेटा, नहीं, मैं अल्लाह नहीं"" हुँ . 🌷लड़का फिर मुस्कराया और कहा, ""तो फिर ज़रूर अल्लाह के दोस्त होंगे, . 🌸क्योंकि मैंने कल रात अल्लाह से कहा था कि मुझे नऐ जूते देदें"". . ☘वो आदमी मुस्कुरा दिया और उसके सर को प्यार से चूमकर अपने घर की तरफ़ चल पड़ा. . ☘अब वो आदमी भी जान चुका था कि अल्लाह का दोस्त होना कोई मुश्किल काम नहीं.. . 🌺खुशियाँ बाटने से मिलती है 🌴 ""सूरेह यासीन"" -- फज्र के बाद पढ़ने से हर ख्वाहिश पूरी होती हे 🌲""सूरेह वाक़िया"" -- मगरिब के बाद पढ़ने से कभी फाका नहीं आता 🍀""सूरेह कौसर "" -- दुश्मनो की दुश्मनी से बचाती हे 🌸""सूरेह काफिरुंन"" -- मौत के वक़्त कुफ़्र से बचाती हे 🍁""सूरेह इखलास"" -- मुनाफ़िक़त से बचाती है 🌼""सूरेह फलक"" -- हादसों से बचाती हे 🌻""सूरेह नास"" -- वसवसो से बचाती हे 🌿ये तोहफा दुसरो को भी दे क्योंकि अल्लाह अज़्ज़ओ जल तोहफा देने वालो को पसंद फरमाता हे.. 💐💐💐💐💐 छोटी सी दुवा ""ए-अल्लाह"" जब तू अपनी रहमतो के दरवाजे खोले और पुकारे... ""है कोई रहमत मांगने वाला.? है कोई खुशिया मांगने वाला.? है कोई शिफा मांगने वाला.? है कोई मेरे ""महबूब"" (सल्ललाहो अलैही वसल्लम)"" की चाहत मांगने वाला.? तो मेरी दुआ है की, ""या अल्लाह"" सारी खुशियाँ सारी रहमते सारी कामयाबियाँ सारी बरकतें सारी नेअ़मतें ""नबी - ए - पाक"" ( सल्ललाहो अलैही वसल्लम )"" की चाहत उस शख्स को देदे जो ये मेसेज पढ़ के दुसरे को भी इस खूबसूरत दुआ में शामिल करे... *ईल्म -ए- ग़ैब चैनल* *यु ट्यूब चैनल क़ो सब्सक्राइब करें* लिंकः https://www.youtube.com/channel/UC2g347mT_09CX92XOG7eRDg *जज़्ज़ाकल्लाह*",1549564858000,None,https://www.youtube.com/channel/UC2g347mT_09CX92XOG7eRDg,0,353,hi,0,0,"पैगाम एक बच्चा जला देने वाली गर्मी में नंगे पैर गुलदस्ते बेच रहा था लोग उसमे भी मोलभाव कर रहे थे एक आदमी को उसके पैर देखकर बहुत दुःख हुआ उस आदमी ने बाज़ार से नया जूता ख़रीदा और उसे देते हुए कहा ""बेटा लो ये जूता पहन लो"" लड़के ने फ़ौरन जूते निकाले और पहन लिए उसका चेहरा ख़ुशी से दमक उठा था वो उस आदमी की तरफ़ पल्टा और हाथ थाम कर पूछा ""आप अल्लाह हैं ""उसने घबरा कर हाथ छुड़ाया और कानों को हाथ लगा कर कहा ""नहीं बेटा नहीं मैं अल्लाह नहीं"" हुँ लड़का फिर मुस्कराया और कहा ""तो फिर ज़रूर अल्लाह के दोस्त होंगे क्योंकि मैंने कल रात अल्लाह से कहा था कि मुझे नऐ जूते देदें"" वो आदमी मुस्कुरा दिया और उसके सर को प्यार से चूमकर अपने घर की तरफ़ चल पड़ा अब वो आदमी भी जान चुका था कि अल्लाह का दोस्त होना कोई मुश्किल काम नहीं खुशियाँ बाटने से मिलती है ""सूरेह यासीन"" फज्र के बाद पढ़ने से हर ख्वाहिश पूरी होती हे ""सूरेह वाक़िया"" मगरिब के बाद पढ़ने से कभी फाका नहीं आता ""सूरेह कौसर "" दुश्मनो की दुश्मनी से बचाती हे ""सूरेह काफिरुंन"" मौत के वक़्त कुफ़्र से बचाती हे ""सूरेह इखलास"" मुनाफ़िक़त से बचाती है ""सूरेह फलक"" हादसों से बचाती हे ""सूरेह नास"

In [118]:
thresh=0.95
fear_speech_label=[]

for index,row in tqdm_notebook(whatsapp_data_pred.iterrows(),total=len(whatsapp_data_pred)):
    flag=0
#     for i in range(1,6):
    if(row['fold_500']>=thresh):
            flag+=1
            
    fear_speech_label.append(flag)

/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [119]:
whatsapp_data_pred['fear_speech_label']=fear_speech_label

In [120]:
pd.set_option('display.max_colwidth', -1)

whatsapp_data_pred[whatsapp_data_pred['fear_speech_label']==1][['message_text','fold_500']]

/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


message_text  \
0       राहुल गांधी मुसलमान हैं केजरीवाल बेईमान हैं मायावती जातिवादी हैं ओर चौकीदार ही चोर हैँ! ठीक                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [95]:
whatsapp_data_pred[whatsapp_data_pred['fear_speech_label']>1][['message_text','fold_1','fold_2','fold_3','fold_4','fold_5']].sample(30).to_csv('Fear_speech_samples.csv')

In [ ]:
whatsapp_data_pred[whatsapp_data_pred['fold_1']>0.9].sample(1).iloc[0]['message_text']

In [ ]:
whatsapp_data_pred.loc[127]

In [ ]:
for i in range(0,1000,5):
    print(i)

In [99]:
from collections import OrderedDict 
group_dict={}
for group in data_fear_speech.phone_num_anonymized.unique():
    temp=data_fear_speech[data_fear_speech['phone_num_anonymized']==group]
    group_dict[group]=len(temp)
dict1 = sorted(group_dict.items(),key=lambda item: item[1],reverse=True)

selected_groups=[ele[0] for ele in dict1[0:100]]
len(selected_groups)

100

In [101]:
temp=whatsapp_data_pred[whatsapp_data_pred['phone_num_anonymized'].isin(selected_groups)]['message_text']

In [109]:
temp.to_csv(parent_path+'Fear_speech_user.csv',index=False)

15649

,group_id_anonymized,phone_num_anonymized,message_text,timestamp,media_type,media_url,forwarded,orig_index,language,only_urls,is_spam,preprocessed,hindu_phrases_count,muslim_phrases_count,fold_1,fold_2,fold_3,fold_4,fold_5,fear_speech_label
